In [1]:
#import modules
from numpy import NaN
import requests
import json
import pandas as pd
import geopandas as gpd
from pprint import pprint
import os.path
import sys
import time
import random

In [4]:
#set study area
#load ooutput area file for London
ons_oa = gpd.read_file('../../data/boundary_london/kc_2011_oa.geojson')
#extract oa for kensington and chelsea only
kc_oa = ons_oa[ons_oa['LAD16NM'] == 'Kensington and Chelsea']
#id column
kc_oa_id_column = "OA11CD"

In [5]:
#extract osm data for kc by each oa
list_df = []
for this_oa_id in kc_oa[kc_oa_id_column]:
    this_section = kc_oa[kc_oa[kc_oa_id_column] == this_oa_id]
    print("Working on %s:" % (this_oa_id))
    print(this_section)
    this_section_geojson = this_section.to_json()
    #query latest  data for Kenisngton and Chelsea 
    times_to_query = '2021-10-15'
    URL = 'https://api.ohsome.org/v1/elements/geometry'
    data = {"bpolys": this_section_geojson, "time": times_to_query, "properties":'tags', "filter": "type:node"}
    response = requests.post(URL, data=data)
    response_dict = response.json()
    #print(response.json())
    oa_data = pd.json_normalize(response_dict, record_path =['features'])
    list_df.append(oa_data)
#add dataframes
kc_osm_data = pd.concat(list_df) 
print(kc_osm_data)
kc_osm_data.to_csv('../../data/data_ohsome/new_kc_osm.csv')
#get column names
print(kc_osm_data.columns)

Working on E00014043:
   OBJECTID     OA11CD    LAD11CD    LAD16CD                 LAD16NM  \
0     13669  E00014043  E09000020  E09000020  Kensington and Chelsea   

                               GlobalID  \
0  9cb5575c-d471-4f55-9d15-246e424bdbef   

                                            geometry  
0  POLYGON ((-0.19268 51.49957, -0.19286 51.49954...  
Working on E00014044:
   OBJECTID     OA11CD    LAD11CD    LAD16CD                 LAD16NM  \
1     13670  E00014044  E09000020  E09000020  Kensington and Chelsea   

                               GlobalID  \
1  4e53997c-0ab8-4958-ac5f-3ea984d566e4   

                                            geometry  
1  POLYGON ((-0.19524 51.49862, -0.19524 51.49861...  
Working on E00014045:
   OBJECTID     OA11CD    LAD11CD    LAD16CD                 LAD16NM  \
2     13671  E00014045  E09000020  E09000020  Kensington and Chelsea   

                               GlobalID  \
2  8975649f-a925-435c-9840-4e8c650dc42d   

                  

In [6]:
kc_osm_data.to_csv('../../data/data_ohsome/kc_osm_agile_testing.csv')